<a href="https://colab.research.google.com/github/yashgadiwan/skill/blob/main/skill%207%2C8%2C9%2C10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

exp 7-8

In [1]:
#pearsons

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
file_path = "/content/archive.csv"  # Update path if needed
df = pd.read_csv(file_path)

# Define numerical features and target column
target_column = 'Starting_Salary'  # Replace with actual numeric target if different
numerical_features = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Check if the target column exists in your DataFrame
if target_column not in df.columns:
    raise KeyError(f"The target column '{target_column}' is not found in the DataFrame.")

# Compute Pearson correlation for each feature with target
correlation_results = {}

print("\n===== Pearson Correlation Test =====")
for feature in numerical_features:
    if feature != target_column:
        corr, _ = pearsonr(df[feature], df[target_column])
        correlation_results[feature] = corr
        print(f"Feature: {feature}, Correlation: {corr}")

# Apply threshold-based filtering
threshold = 0.2  # Change this as needed (absolute correlation threshold)
filtered_features = {k: v for k, v in correlation_results.items() if abs(v) >= threshold}

# Display filtered features
print("\n===== Features Above Correlation Threshold =====")
for feature, corr in filtered_features.items():
    print(f"Feature: {feature}, Correlation: {corr}")

# Create correlation matrix
correlation_matrix = df[numerical_features].corr()

# Display correlation matrix
print("\n===== Correlation Matrix =====")
print(correlation_matrix)

# Plot heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
plt.title("Feature Correlation Matrix")
plt.show()

ANOVA

In [ ]:


import pandas as pd
import numpy as np
from sklearn.feature_selection import f_classif

# Define numerical features and target variable
numerical_features = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
target_column = 'Job_Offers'

# Ensure the target column is numeric (ANOVA requires numeric target categories)
df[target_column] = pd.to_numeric(df[target_column])

# Perform ANOVA F-Test
f_scores, p_values = f_classif(df[numerical_features], df[target_column])

# Display feature scores
print("\n===== ANOVA Feature Selection Results =====")
for feature, score, p_value in zip(numerical_features, f_scores, p_values):
    print(f"Feature: {feature}, Score: {score:.2f}, p-value: {p_value:.4f}")

# Optional: Filter features with a p-value less than 0.05 (statistically significant)
significant_features = [feature for feature, p in zip(numerical_features, p_values) if p < 0.05]

print("\nStatistically Significant Features (p < 0.05):")
print(significant_features)

CHI- SQUARE

In [ ]:


import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest, chi2

# Define categorical features and target variable
categorical_features = ['Gender', 'Field_of_Study', 'Current_Job_Level', 'Entrepreneurship']
target_column = 'Job_Offers'  # Ensure this is a categorical variable

# Convert categorical features into numerical using one-hot encoding
df_encoded = pd.get_dummies(df[categorical_features])

# Perform Chi-Square test using SelectKBest
k = 5  # Number of top features to select
selector = SelectKBest(score_func=chi2, k='all')  # Initially select all
X_new = selector.fit_transform(df_encoded, df[target_column])

# Get feature scores
feature_scores = selector.scores_
threshold = np.percentile(feature_scores, 50)  # Select features above median score

# Get top-K features
top_k_selector = SelectKBest(score_func=chi2, k=k)
X_k_new = top_k_selector.fit_transform(df_encoded, df[target_column])
selected_k_features = np.array(df_encoded.columns)[top_k_selector.get_support()]
top_k_scores = top_k_selector.scores_[top_k_selector.get_support()]

# Display results for threshold-based selection
print("\n===== Chi-Square Features (Above Threshold) =====")
for feature, score in zip(df_encoded.columns, feature_scores):
    if score >= threshold:
        print(f"Feature: {feature}, Score: {score}")

# Display results for top-K selection
print("\n===== Top-K Chi-Square Features =====")
for feature, score in zip(selected_k_features, top_k_scores):
    print(f"Feature: {feature}, Score: {score}")

i g

In [ ]:


import pandas as pd
import numpy as np
from sklearn.feature_selection import mutual_info_classif, SelectKBest

# Define target variable
target_column = 'Career_Satisfaction'  # Replace with actual categorical target

# Encode categorical target
df[target_column] = df[target_column].astype('category').cat.codes

# Select numerical features
numerical_features = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Compute mutual information (information gain)
mi_scores = mutual_info_classif(df[numerical_features], df[target_column])

# Apply threshold-based filtering
threshold = np.percentile(mi_scores, 50)  # Select features above the 50th percentile

# Get top-K features
k = 5  # Number of top features to select
top_k_selector = SelectKBest(score_func=mutual_info_classif, k=k)
X_k_new = top_k_selector.fit_transform(df[numerical_features], df[target_column])
selected_k_features = np.array(numerical_features)[top_k_selector.get_support()]
top_k_scores = top_k_selector.scores_[top_k_selector.get_support()]

# Display results for threshold-based selection
print("\n===== Information Gain (Above Threshold) =====")
for feature, score in zip(numerical_features, mi_scores):
    if score >= threshold:
        print(f"Feature: {feature}, Information Gain: {score}")

# Display results for top-K selection
print("\n===== Top-K Information Gain Features =====")
for feature, score in zip(selected_k_features, top_k_scores):
    print(f"Feature: {feature}, Information Gain: {score}")

experiment 9-10

forward selection

In [ ]:

import pandas as pd
import statsmodels.api as sm
from sklearn.preprocessing import LabelEncoder

# Load data
file_path = "/content/archive.csv"  # Update path if needed
df = pd.read_csv(file_path)

# Define features and target
X = df.drop('Job_Offers', axis=1)
y = df['Job_Offers']

# Encode categorical columns
for col in X.select_dtypes(include='object').columns:
    X[col] = LabelEncoder().fit_transform(X[col])

# Forward Selection
selected_features = []
remaining_features = list(X.columns)

for _ in range(len(remaining_features)):
    best_score = float('-inf')
    best_feature = None

    for feature in remaining_features:
        trial_features = selected_features + [feature]
        X_trial = sm.add_constant(X[trial_features])
        model = sm.OLS(y, X_trial).fit()

        if model.rsquared > best_score:
            best_score = model.rsquared
            best_feature = feature

    if best_feature is not None:
        selected_features.append(best_feature)
        remaining_features.remove(best_feature)

print("Selected Features (Forward Selection):", selected_features)



backward Elimination.

In [ ]:


import statsmodels.api as sm

# Start with all features
features = list(X.columns)
X_with_const = sm.add_constant(X[features])

while len(features) > 0:
    model = sm.OLS(y, X_with_const).fit()
    p_values = model.pvalues[1:]

    worst_p = p_values.idxmax()
    if p_values[worst_p] > 0.05:
        features.remove(worst_p)
        X_with_const = sm.add_constant(X[features])
    else:
        break

print("Selected Features (Backward Elimination):", features)

Recursive Feature Elimination (RFE):

In [ ]:


from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()
rfe = RFE(model, n_features_to_select=5)
fit = rfe.fit(X, y)

selected_features = X.columns[fit.support_].tolist()
print("Selected Features (RFE):", selected_features)

Cross Validation (CV):

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()
cv_scores = cross_val_score(model, X, y, cv=5, scoring='r2')

print("Cross-Validation Scores:", cv_scores)
print("Average CV Score:", cv_scores.mean())